In [100]:
import pandas as pd
import numpy as np
import os
import googlemaps
import gmaps
from pprint import pprint
import openrouteservice
from ipywidgets.embed import embed_minimal_html
import time
import folium
import requests
import geopy
from geopy.geocoders import Nominatim
import selenium

In [110]:
API_KEY = open('api_keys/API_KEY.txt').read()
map_client = googlemaps.Client(API_KEY)

In [3]:
or_API_KEY = open('api_keys/openrouteAPI.txt').read()
or_client = openrouteservice.Client(key = or_API_KEY)

In [4]:
location_name = "food truck indianapolis"
result_list = []

In [5]:
response = map_client.places(query=location_name)
pg2_token = response['next_page_token']
result = response.get('results')

In [6]:
response2 = map_client.places(page_token=pg2_token)
pg3_token = response2['next_page_token']
result2 = response2.get('results')

In [7]:
response3 = map_client.places(page_token=pg3_token)
try:
    pg4_token = response3['next_page_token']
except KeyError:
    print('no more pages')
result3 = response3.get('results')

no more pages


In [8]:
result_list.extend([result,result2,result3])

In [9]:
def get_sat_hours(all_hours):
    for i in range(len(all_hours)):
        day = all_hours[i]['close']['day']
        if day == 5:
            sat_open = military_to_us(int(all_hours[i]['open']['time']))
            sat_close = military_to_us(int(all_hours[i]['close']['time']))
        else:
            continue
    return sat_open, sat_close
def get_sun_hours(all_hours):
    for i in range(len(all_hours)):
        day = all_hours[i]['close']['day']
        if day == 6:
            sun_open = military_to_us(int(all_hours[i]['open']['time']))
            sun_close = military_to_us(int(all_hours[i]['close']['time']))
        else:
            continue
    return sun_open, sun_close
def military_to_us(time):
    mark = 'AM'
    if str(time) == '0':
        return '12:00AM'
    if time == 1200:
        return '12:00PM'
    if time>1200:
        time = time - 1200
        mark = 'PM'
    if time >= 1000:
        time = str(time)[:2] + ":" + str(time)[2:] + mark
    else:
        time = str(time)[:1] + ":" + str(time)[1:] + mark
    return time

In [10]:
names = [] # name of food truck
address = [] # formatted address of food truck
ratings = [] # rating out of 5 for food truck
saturday_oc = [] # open or closed on saturday
sunday_oc = [] # open or closed on sunday
sat_open = [] # open hour for saturday if open
sat_close = [] # close hour for saturday if open
sun_open = [] # open hour for sunday if open
sun_close = [] # close hour for sunday if open
urls = [] # food truck website, if they don't have a website google maps url is used
bus_status = [] # operational, temporarily closed
serve_beer = [] # True/false
serve_wine = [] # True/false
serve_lunch = [] # True/false
serve_dinner = [] # True/false
latitude = [] # latitude for food truck location
longitude = [] # longitude for food truck location
# loop through each result from original API call to create lists with desired information for each result
for j in range(len(result_list)):
    for i in range(len(result_list[j])):
        names.append(result_list[j][i]['name'])
        address.append(result_list[j][i]['formatted_address'])
        ratings.append(result_list[j][i]['rating'])
        latitude.append(result_list[j][i]['geometry']['location']['lat'])
        longitude.append(result_list[j][i]['geometry']['location']['lng'])
        place_id = result_list[j][i]['place_id']
        place_details = map_client.place(place_id)
        details_results = place_details.get('result')
        bus_status.append(details_results['business_status'])
        try:
            serve_beer.append(details_results['serves_beer'])
        except:
            serve_beer.append('n/a')
        try:
            serve_dinner.append(details_results['serves_dinner'])
        except:
            serve_dinner.append('n/a')
        try:
            serve_lunch.append(details_results['serves_lunch'])
        except:
            serve_lunch.append('n/a')
        try:
            serve_wine.append(details_results['serves_wine'])
        except:
            serve_wine.append('n/a')
            
        try:
            if (details_results['opening_hours']['weekday_text'][5] == "Saturday: Closed"):
                saturday_oc.append("Closed")
                sat_open.append("Closed")
                sat_close.append("Closed")
            else:
                sat_hours = get_sat_hours(details_results['opening_hours']['periods'])
                saturday_oc.append("Open")
                sat_open.append(sat_hours[0])
                sat_close.append(sat_hours[1])
            if (details_results['opening_hours']['weekday_text'][6] == "Sunday: Closed"):
                sunday_oc.append("Closed")
                sun_open.append("Closed")
                sun_close.append("Closed")
            else:
                sun_hours = get_sat_hours(details_results['opening_hours']['periods'])
                sunday_oc.append("Open")
                sun_open.append(sun_hours[0])
                sun_close.append(sun_hours[1])
        except:
            saturday_oc.append('n/a')
            sunday_oc.append('n/a')
            sat_open.append("n/a")
            sat_close.append("n/a")
            sun_open.append("n/a")
            sun_close.append("n/a")
        try: 
            url = details_results['website']
            urls.append(url)
        except:
            urls.append(details_results['url'])
# puts lists into dictionary for easy conversion to dataframe
food_truck_dict = {'name': names, 'address': address, 'rating':ratings, 'sat_open': saturday_oc, 'sun_open':sunday_oc, 'sat_open_hr':sat_open, 'sat_close_hr':sat_close, 
                   'sun_open_hr': sun_open, 'sun_close_hr': sun_close, 'serve_lunch': serve_lunch, 'serve_dinner': serve_dinner, 'serve_beer': serve_beer, 'serve_wine': serve_wine,
                   'website_url':urls, 'business_status': bus_status, 'latitude': latitude, 'longitude': longitude}
# dictionary to data frame
food_truck_df = pd.DataFrame(food_truck_dict)

In [11]:
food_truck_df

,name,address,rating,sat_open,sun_open,sat_open_hr,sat_close_hr,sun_open_hr,sun_close_hr,serve_lunch,serve_dinner,serve_beer,serve_wine,website_url,business_status,latitude,longitude
0,big Jerry Indy food truck,"8522 Madison Ave, Indianapolis, IN 46227, Unit...",5.0,Open,Closed,10:30AM,7:00PM,Closed,Closed,True,True,n/a,n/a,https://maps.google.com/?cid=18195972309441806285,OPERATIONAL,39.642731,-86.120180
1,Pico de Gallo Food Truck,"3014 Lafayette Rd #3000, Indianapolis, IN 4622...",4.5,Open,Open,5:00PM,1:00AM,5:00PM,1:00AM,True,True,False,False,https://www.facebook.com/peregrino1982/,OPERATIONAL,39.809920,-86.221209
2,The Latin Flavor - Food Truck,"9130 E 38th St, Indianapolis, IN 46235, United...",4.1,Open,Open,11:00AM,11:00PM,11:00AM,11:00PM,True,True,n/a,n/a,http://thelatinflavorindy.com/?utm_source=gmb&...,OPERATIONAL,39.826405,-86.006841
3,Mi Lindo Pénjamo - Food Truck,"7910 Michigan Rd, Indianapolis, IN 46268, Unit...",4.5,Closed,Closed,Closed,Closed,Closed,Closed,True,True,n/a,n/a,https://store.kahero.co/milindopenjamo,OPERATIONAL,39.897543,-86.216448
4,Taco food truck,"5621 N Parker Ave, Indianapolis, IN 46220, Uni...",0.0,n/a,n/a,n/a,n/a,n/a,n/a,True,True,n/a,n/a,https://maps.google.com/?cid=8721236106944255517,OPERATIONAL,39.855708,-86.113707
5,Super Tacos food truck,"7202 E Washington St, Indianapolis, IN 46219, ...",4.6,n/a,n/a,n/a,n/a,n/a,n/a,True,True,n/a,n/a,https://maps.google.com/?cid=798189214973396941,OPERATIONAL,39.772545,-86.040350
6,Antojitos Guzman food truck #1,"4853 W 56th St, Indianapolis, IN 46254, United...",3.5,Open,Open,2:00PM,9:30PM,2:00PM,9:30PM,True,True,n/a,n/a,https://antojitosguzmanfoodtruck1.business.sit...,OPERATIONAL,39.852722,-86.241436
7,Emergency Taco Truck,"7001 Pendleton Pike, Indianapolis, IN 46226, U...",4.7,n/a,n/a,n/a,n/a,n/a,n/a,True,True,False,False,https://maps.google.com/?cid=5987304690148734817,OPERATIONAL,39.827233,-86.045250
8,Food truck Mil Amores,"5034 E Raymond St, Indianapolis, IN 46203, Uni...",4.0,n/a,n/a,n/a,n/a,n/a,n/a,True,True,n/a,n/a,https://maps.google.com/?cid=12871198745142313123,OPERATIONAL,39.738616,-86.082196
9,ClusterTruck,"729 N Pennsylvania St, Indianapolis, IN 46204,...",4.8,Open,Open,8:00AM,11:00PM,8:00AM,11:00PM,n/a,n/a,n/a,n/a,https://downtown-indy.clustertruck.com/menu?ut...,OPERATIONAL,39.777586,-86.155602


## Average Food Truck Rating for Trucks open on Saturday

In [16]:
print("Average rating = ",round(food_truck_df[food_truck_df['sat_open'] == "Open"]['rating'].mean(),2))

Average rating =  4.36


## Times Saturday Food Trucks Open 

In [11]:
food_truck_df['sat_open_hr'].value_counts()

sat_open_hr
n/a        19
11:00AM     8
12:00PM     5
6:00PM      5
10:00AM     5
Closed      4
5:00PM      3
10:30AM     2
8:00AM      2
2:00PM      1
2:30PM      1
8:30PM      1
1:59PM      1
3:00PM      1
4:00PM      1
7:00PM      1
Name: count, dtype: int64

## Average Food Truck Rating for Trucks open on Saturday

In [18]:
print("Average rating = ",round(food_truck_df[food_truck_df['sun_open'] == "Open"]['rating'].mean(),2))

Average rating =  4.29


## Times Saturday Food Trucks Open 

In [21]:
food_truck_df['sun_open_hr'].value_counts()

sun_open_hr
n/a        19
Closed     17
11:00AM     4
6:00PM      4
10:00AM     4
5:00PM      3
2:00PM      1
8:00AM      1
2:30PM      1
8:30PM      1
10:30AM     1
12:00PM     1
1:59PM      1
3:00PM      1
7:00PM      1
Name: count, dtype: int64

As you can see here 13 more restauants are closed on Sunday. 7 of these 13 are restaurants that open up in the morning. This could be due to a variety of reasons one being that the food trucks owners taking a day off for church and family. Another possible reason could be that they are closing because business slows as a lot of other use Sunday for church and regrouping for the next week ahead.

In [22]:
# create subsets of food trucks that would be preferred during different times of the day

# food trucks that are open on saturday mornings
sat_morning = food_truck_df.loc[food_truck_df['sat_open_hr'].str.contains("AM")]

# food trucks that serve lunch are are open on saturdays
sat_lunch = food_truck_df.loc[(food_truck_df['sat_open'] == "Open") & (food_truck_df['serve_lunch'] == True)]

# food trucks that serve dinner and are open on saturday
sat_dinner = food_truck_df.loc[(food_truck_df['sat_open'] == "Open") & (food_truck_df['serve_dinner'] == True)]

#food trucks that are open late saturday night (into early sunday morning)
sat_latenight = food_truck_df.loc[food_truck_df['sat_close_hr'].str.contains("AM")]

#create similar subsets for sunday
# food trucks that are open on sunday morning
sun_morning = food_truck_df.loc[food_truck_df['sun_open_hr'].str.contains("AM")]

# food trucks that serve lunch are are open on sundays
sun_lunch = food_truck_df.loc[(food_truck_df['sun_open'] == "Open") & (food_truck_df['serve_lunch'] == True)]

# food trucks that serve dinner and are open on sundays
sun_dinner = food_truck_df.loc[(food_truck_df['sun_open'] == "Open") & (food_truck_df['serve_dinner'] == True)]

#food trucks that are open late sunday night (into early sunday morning)
sun_latenight = food_truck_df.loc[food_truck_df['sun_close_hr'].str.contains("AM")]

## Function to return a df of 8 food trucks to visit throughout the weekend

#### Can pass in True/False to drinks wanted if you want to be able to order drinks at dinner (default = False)

#### Can pass in minimum preferred rating of the food trucks you visit (default = 3.0)

In [53]:
def get_weekend_food_trucks(sat_morning = sat_morning, sat_lunch = sat_lunch, sat_dinner = sat_dinner, sat_latenight = sat_latenight, 
                            sun_morning = sun_morning, sun_lunch = sun_lunch, sun_dinner = sun_dinner, sun_latenight = sun_latenight, 
                            drinks_wanted = False, preferred_rating = 3.0):
    schedule = pd.DataFrame(columns = sat_morning.columns)
    if (drinks_wanted == True):
        sat_dinner = sat_dinner[(sat_dinner['serve_beer'] == True) | (sat_dinner['serve_wine'] == True)]
        sun_dinner = sun_dinner[(sun_dinner['serve_beer'] == True) | (sun_dinner['serve_wine'] == True)]
    
    sat_morning = sat_morning[sat_morning['rating'] >= preferred_rating]
    schedule = pd.concat([schedule,sat_morning.sample()])
    sat_lunch = sat_lunch[(sat_lunch['rating'] >= preferred_rating) & (~sat_lunch['name'].isin(schedule['name']))]
    schedule = pd.concat([schedule,sat_lunch.sample()])
    sat_dinner = sat_dinner[(sat_dinner['rating'] >= preferred_rating) & (~sat_dinner['name'].isin(schedule['name']))]
    schedule = pd.concat([schedule,sat_dinner.sample()])
    sat_latenight = sat_latenight[(sat_latenight['rating'] >= preferred_rating) & (~sat_latenight['name'].isin(schedule['name']))]
    schedule = pd.concat([schedule,sat_latenight.sample()])
    
    sun_morning = sun_morning[(sun_morning['rating'] >= preferred_rating) & (~sun_morning['name'].isin(schedule['name']))]
    schedule = pd.concat([schedule,sun_morning.sample()])
    sun_lunch = sun_lunch[(sun_lunch['rating'] >= preferred_rating) & (~sun_lunch['name'].isin(schedule['name']))]
    schedule = pd.concat([schedule,sun_lunch.sample()])
    sun_dinner = sun_dinner[(sun_dinner['rating'] >= preferred_rating) & (~sun_dinner['name'].isin(schedule['name']))]
    schedule = pd.concat([schedule,sun_dinner.sample()])
    sun_latenight = sun_latenight[(sun_latenight['rating'] >= preferred_rating) & (~sun_latenight['name'].isin(schedule['name']))]
    schedule = pd.concat([schedule,sun_latenight.sample()])
    
    days = ['Saturday', 'Sunday']
    days = np.repeat(days,4)
    
    schedule['visit_day'] = days
    
    return schedule

## Create Weekend Schedule

#### I'll create 3 for examples

In [54]:
schedule1 = get_weekend_food_trucks()
schedule2 = get_weekend_food_trucks(preferred_rating=4.5)
schedule3 = get_weekend_food_trucks(drinks_wanted=True, preferred_rating=4.0)

#### Get specific line ups for saturday and sunday

In [55]:
saturday1 = schedule1[schedule1['visit_day'] == 'Saturday']
sunday1 = schedule1[schedule1['visit_day'] == 'Sunday']

saturday2 = schedule2[schedule2['visit_day'] == 'Saturday']
sunday2 = schedule2[schedule2['visit_day'] == 'Sunday']

saturday3 = schedule3[schedule3['visit_day'] == 'Saturday']
sunday3 = schedule3[schedule3['visit_day'] == 'Sunday']

In [73]:
saturday1

,name,address,rating,sat_open,sun_open,sat_open_hr,sat_close_hr,sun_open_hr,sun_close_hr,serve_lunch,serve_dinner,serve_beer,serve_wine,website_url,business_status,latitude,longitude,visit_day
34,ClusterTruck - Castleton,"4737 E 82nd St, Indianapolis, IN 46250, United...",4.9,Open,Closed,11:00AM,9:00PM,Closed,Closed,True,True,False,False,http://clustertruck.com/,OPERATIONAL,39.904795,-86.089632,Saturday
41,Taqueria Mi Costa,"1430 W Washington St, Indianapolis, IN 46222, ...",4.3,Open,Open,11:00AM,11:00PM,11:00AM,11:00PM,True,True,n/a,n/a,https://taqueria-mi-costa.business.site/,OPERATIONAL,39.767596,-86.186622,Saturday
28,Fiesta on The Go,"2719 E 38th St, Indianapolis, IN 46218, United...",4.0,Open,Closed,4:00PM,10:00PM,Closed,Closed,True,True,False,False,https://maps.google.com/?cid=11107978367519840184,OPERATIONAL,39.825104,-86.117761,Saturday
21,Food Truck Don Luis,"3605 Commercial Dr, Indianapolis, IN 46222, Un...",3.0,Open,Open,6:00PM,12:00AM,6:00PM,12:00AM,n/a,n/a,n/a,n/a,http://orderfoodtruckdonluis.com/,OPERATIONAL,39.818318,-86.226983,Saturday


#### Designate start and stop location. This is where the route will begin and end each day for the tour
For this example I will just use IUPUI Campus Center. More than likely this would be an apartment or hotel

In [56]:
start_stop_location = "IUPUI Campus Center"

#### Functions to get walking and driving directions. 
Information will be used later to decide whether to walk or drive between locations

In [30]:
def get_walk_results(df):
    results = map_client.directions(origin = start_stop_location,
                                         destination = start_stop_location,                                     
                                         waypoints = list(df['address']),
                                         mode='walking')
    time.sleep(2)
    return results

def get_drive_results(df):
    results = map_client.directions(origin = start_stop_location,
                                         destination = start_stop_location,                                     
                                         waypoints = list(df['address']),
                                         mode='driving')
    time.sleep(2)
    return results

In [57]:
sat1_walk_results = get_walk_results(saturday1)
sat1_drive_results = get_drive_results(saturday1)
sun1_walk_results = get_walk_results(sunday1)
sun1_drive_results = get_drive_results(sunday1)
sat2_walk_results = get_walk_results(saturday2)
sat2_drive_results = get_drive_results(saturday2)
sun2_walk_results = get_walk_results(sunday2)
sun2_drive_results = get_drive_results(sunday2)
sat3_walk_results = get_walk_results(saturday3)
sat3_drive_results = get_drive_results(saturday3)
sun3_walk_results = get_walk_results(sunday3)
sun3_drive_results = get_drive_results(sunday3)

## Function to get dataframe with information for deciding whether to walk or drive each leg

In [58]:
def decide_method(row):
    if float(row['walk_distance'][:-3]) <= 1.25:
        method = "walk"
    else:
        method = "drive"
    return method

In [92]:
def get_saturday_decide_df(df_walk, df_drive, saturday_df):
    start_location = []
    start_address = [] 
    end_location = []
    end_address = []
    walk_distance_mi = []
    walk_time_hr_min = []
    walk_time_s = []
    drive_distance_mi = []
    drive_time_hr_min = []
    drive_time_s = []
    leg = []
    rating = []
    open_close = []
    beer = []
    wine = []
    for i in range(len(df_walk[0]['legs'])):
        leg.append(i+1)
        if (i == 0):
            start_location.append(start_stop_location)
            end_location.append(saturday_df['name'].iloc[i])
            rating.append(saturday_df['rating'].iloc[i])
            open_close.append('-'.join([saturday_df['sat_open_hr'].iloc[i],saturday_df['sat_close_hr'].iloc[i]]))
            beer.append(saturday_df['serve_beer'].iloc[i])
            wine.append(saturday_df['serve_wine'].iloc[i])
        elif (i == len(df_walk[0]['legs'])-1):
            end_location.append(start_stop_location)
            start_location.append(saturday_df['name'].iloc[i-1])
            rating.append('-')
            open_close.append('-')
            beer.append('-')
            wine.append('-')
        else:
            start_location.append(saturday_df['name'].iloc[i-1])
            end_location.append(saturday_df['name'].iloc[i])
            rating.append(saturday_df['rating'].iloc[i])
            open_close.append('-'.join([saturday_df['sat_open_hr'].iloc[i],saturday_df['sat_close_hr'].iloc[i]]))
            beer.append(saturday_df['serve_beer'].iloc[i])
            wine.append(saturday_df['serve_wine'].iloc[i])
            
        start_address.append(df_walk[0]['legs'][i]['start_address'])
        end_address.append(df_walk[0]['legs'][i]['end_address'])
        walk_distance_mi.append(df_walk[0]['legs'][i]['distance']['text'])
        walk_time_hr_min.append(df_walk[0]['legs'][i]['duration']['text'])
        walk_time_s.append(df_walk[0]['legs'][i]['duration']['value'])
        drive_distance_mi.append(df_drive[0]['legs'][i]['distance']['text'])
        drive_time_hr_min.append(df_drive[0]['legs'][i]['duration']['text'])
        drive_time_s.append(df_drive[0]['legs'][i]['duration']['value'])

    saturday_decide_dict = {'leg': leg, 'start_location': start_location, 'start_address': start_address, 'end_location':end_location, 'end_address': end_address, 
                            'walk_distance':walk_distance_mi, 'walk_time':walk_time_hr_min, 'walk_time_s':walk_time_s, 'drive_distance':drive_distance_mi,
                            'drive_time':drive_time_hr_min, 'drive_time_s': drive_time_s, 'end_location_rating':rating, 'hours': open_close, 
                            'serve_beer': beer, 'serve_wine': wine}
    # dictionary to data frame
    saturday_decide_df = pd.DataFrame(saturday_decide_dict)   
    saturday_decide_df['method'] = saturday_decide_df.apply(decide_method, axis=1)
    
    return saturday_decide_df

def get_sunday_decide_df(df_walk, df_drive, sunday_df):
    start_location = []
    start_address = [] 
    end_location = []
    end_address = []
    walk_distance_mi = []
    walk_time_hr_min = []
    walk_time_s = []
    drive_distance_mi = []
    drive_time_hr_min = []
    drive_time_s = []
    leg = []
    rating = []
    open_close = []
    beer = []
    wine = []
    for i in range(len(df_walk[0]['legs'])):
        leg.append(i+1)
        if (i == 0):
            start_location.append(start_stop_location)
            end_location.append(sunday_df['name'].iloc[i])
            rating.append(sunday_df['rating'].iloc[i])
            open_close.append('-'.join([sunday_df['sat_open_hr'].iloc[i],sunday_df['sat_close_hr'].iloc[i]]))
            beer.append(sunday_df['serve_beer'].iloc[i])
            wine.append(sunday_df['serve_wine'].iloc[i])
        elif (i == len(df_walk[0]['legs'])-1):
            end_location.append(start_stop_location)
            start_location.append(sunday_df['name'].iloc[i-1])
            rating.append("n/a")
            open_close.append('-')
            beer.append('-')
            wine.append('-')
        else:
            start_location.append(sunday_df['name'].iloc[i-1])
            end_location.append(sunday_df['name'].iloc[i])
            rating.append(sunday_df['rating'].iloc[i])
            open_close.append('-'.join([sunday_df['sat_open_hr'].iloc[i],sunday_df['sat_close_hr'].iloc[i]]))
            beer.append(sunday_df['serve_beer'].iloc[i])
            wine.append(sunday_df['serve_wine'].iloc[i])
        start_address.append(df_walk[0]['legs'][i]['start_address'])
        end_address.append(df_walk[0]['legs'][i]['end_address'])
        walk_distance_mi.append(df_walk[0]['legs'][i]['distance']['text'])
        walk_time_hr_min.append(df_walk[0]['legs'][i]['duration']['text'])
        walk_time_s.append(df_walk[0]['legs'][i]['duration']['value'])
        drive_distance_mi.append(df_drive[0]['legs'][i]['distance']['text'])
        drive_time_hr_min.append(df_drive[0]['legs'][i]['duration']['text'])
        drive_time_s.append(df_drive[0]['legs'][i]['duration']['value'])

    sunday_decide_dict = {'leg': leg, 'start_location': start_location, 'start_address': start_address, 'end_location':end_location, 'end_address': end_address, 
                          'walk_distance':walk_distance_mi, 'walk_time':walk_time_hr_min, 'walk_time_s':walk_time_s, 'drive_distance':drive_distance_mi,
                          'drive_time':drive_time_hr_min, 'drive_time_s': drive_time_s, 'end_location_rating':rating,  'hours': open_close, 
                          'serve_beer': beer, 'serve_wine': wine}
    # dictionary to data frame
    sunday_decide_df = pd.DataFrame(sunday_decide_dict)   
    sunday_decide_df['method'] = sunday_decide_df.apply(decide_method, axis=1)
    
    return sunday_decide_df

In [93]:
saturday1_decide_df = get_saturday_decide_df(sat1_walk_results, sat1_drive_results, saturday1)
saturday2_decide_df = get_saturday_decide_df(sat2_walk_results, sat2_drive_results, saturday2)
saturday3_decide_df = get_saturday_decide_df(sat3_walk_results, sat3_drive_results, saturday3)

sunday1_decide_df = get_sunday_decide_df(sun1_walk_results, sun1_drive_results, sunday1)
sunday2_decide_df = get_sunday_decide_df(sun2_walk_results, sun2_drive_results, sunday2)
sunday3_decide_df = get_sunday_decide_df(sun3_walk_results, sun3_drive_results, sunday3)

In [94]:
saturday3_decide_df

,leg,start_location,start_address,end_location,end_address,walk_distance,walk_time,walk_time_s,drive_distance,drive_time,drive_time_s,end_location_rating,hours,serve_beer,serve_wine,method
0,1,IUPUI Campus Center,"420 University Blvd, Indianapolis, IN 46202, USA",Taqueria El Taco Loco,"2705 W Washington St, Indianapolis, IN 46222, USA",2.1 mi,46 mins,2773,2.2 mi,6 mins,376,4.4,11:00AM-11:50PM,True,n/a,drive
1,2,Taqueria El Taco Loco,"2705 W Washington St, Indianapolis, IN 46222, USA",Fiesta on The Go,"2719 E 38th St, Indianapolis, IN 46218, USA",7.7 mi,2 hours 50 mins,10219,10.4 mi,18 mins,1078,4.0,4:00PM-10:00PM,False,False,drive
2,3,Fiesta on The Go,"2719 E 38th St, Indianapolis, IN 46218, USA",Gaucho's Fire Express,"906 Carrollton Ave Suite 320, Indianapolis, IN...",4.0 mi,1 hour 29 mins,5342,4.3 mi,12 mins,747,4.7,10:00AM-10:00PM,True,n/a,drive
3,4,Gaucho's Fire Express,"906 Carrollton Ave Suite 320, Indianapolis, IN...",Restaurante Oaxaca,"5101 W 38th St, Indianapolis, IN 46254, USA",7.4 mi,2 hours 46 mins,9946,8.1 mi,18 mins,1060,4.5,6:00PM-2:00AM,False,False,drive
4,5,Restaurante Oaxaca,"5101 W 38th St, Indianapolis, IN 46254, USA",IUPUI Campus Center,"420 University Blvd, Indianapolis, IN 46202, USA",6.1 mi,2 hours 15 mins,8098,7.4 mi,13 mins,787,-,-,-,-,drive


## Export Schedules to CSVs

In [95]:
def export_schedule(df, name):
    df = df.drop(['walk_time_s','drive_time_s'], axis=1)
    df.to_csv(name, index=False)

In [97]:
export_schedule(saturday1_decide_df,"ex_1_saturday.csv")
export_schedule(saturday2_decide_df,"ex_2_saturday.csv")
export_schedule(saturday3_decide_df,"ex_3_saturday.csv")
export_schedule(sunday1_decide_df,"ex_1_sunday.csv")
export_schedule(sunday2_decide_df,"ex_2_sunday.csv")
export_schedule(sunday3_decide_df,"ex_3_sunday.csv")

## Map Generation

In [61]:
rapid_API_KEY = open('api_keys/rapid_API_KEY.txt').read()
def get_lat_long_from_address(address):
   locator = Nominatim(user_agent='http://localhost:8888/lab',timeout=100)
   location = locator.geocode(address)
   return location.latitude, location.longitude
def get_directions_response(lat1, long1, lat2, long2, mode='drive'):
   url = "https://route-and-directions.p.rapidapi.com/v1/routing"
   key = rapid_API_KEY
   host = "route-and-directions.p.rapidapi.com"
   headers = {"X-RapidAPI-Key": key, "X-RapidAPI-Host": host}
   querystring = {"waypoints":f"{str(lat1)},{str(long1)}|{str(lat2)},{str(long2)}","mode":mode}
   response = requests.request("GET", url, headers=headers, params=querystring)
   return response

In [98]:
def create_map(day_schedule):
    addresses = [day_schedule['start_address'][0]]
    for i in range(len(day_schedule['start_address'])):
        addresses.append(day_schedule['end_address'][i])
    
    lat_lons = []
    for addr in addresses:
        try:
            lat_lons.append(get_lat_long_from_address(addr))
        except:
            print("NOTE: address:", addr," not found")
        time.sleep(3) 
        
    responses = []
    for n in range(len(lat_lons)-1):
        lat1, lon1, lat2, lon2 = lat_lons[n][0], lat_lons[n][1], lat_lons[n+1][0], lat_lons[n+1][1]
        response = get_directions_response(lat1, lon1, lat2, lon2, mode='drive')
        responses.append(response)
        time.sleep(3)
        
    m = folium.Map()
    df = pd.DataFrame()
    # add markers for the places we visit
    for point in lat_lons:
        folium.Marker(point).add_to(m)
    # loop over the responses and plot the lines of the route
    for response in responses:
        mls = response.json()['features'][0]['geometry']['coordinates']
        points = [(i[1], i[0]) for i in mls[0]]
      
        # add the lines
        folium.PolyLine(points, weight=5, opacity=1).add_to(m)
        temp = pd.DataFrame(mls[0]).rename(columns={0:'Lon', 1:'Lat'})[['Lat', 'Lon']]
        df = pd.concat([df, temp])
    # create optimal zoom
    sw = df[['Lat', 'Lon']].min().values.tolist()
    sw = [sw[0]-0.0005, sw[1]-0.0005]
    ne = df[['Lat', 'Lon']].max().values.tolist()
    ne = [ne[0]+0.0005, ne[1]+0.0005]
    m.fit_bounds([sw, ne])
    return m

def save_map_png(m,name):
    img_data = m._to_png(5)
    img = Image.open(io.BytesIO(img_data))
    img.save(name + '.png')

## Example 1 maps

In [69]:
m = create_map(saturday1_decide_df)
m.save('saturday1_map.html')

In [103]:
m = create_map(sunday1_decide_df)
m.save("sunday1_map.html")
m

Process repeats for examples 2 and 3 maps

In [105]:
m = create_map(saturday2_decide_df)
m.save("saturday2_map.html")
m

In [106]:
m = create_map(sunday2_decide_df)
m.save("sunday2_map.html")
m

NOTE: address: 3014 Lafayette Rd #3000, Indianapolis, IN 46222, USA  not found
NOTE: address: 906 Carrollton Ave Suite 320, Indianapolis, IN 46202, USA  not found


In [107]:
m = create_map(saturday3_decide_df)
m.save("saturday3_map.html")
m

NOTE: address: 906 Carrollton Ave Suite 320, Indianapolis, IN 46202, USA  not found


In [108]:
m = create_map(sunday3_decide_df)
m.save("sunday3_map.html")
m

## The End
For now